In [4]:
import requests
import json
import re
import numpy as np
import os, errno
from pathlib import Path

In [ ]:
def create_dir(path):  
    try:
        os.makedirs(path)
        
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    else:
        print("Successfully created the directory %s" % path)

In [ ]:
def fetch_html(url, raw=False):
    print(f'Processing {url}')
    r = requests.get(url)

    if not r.ok:
        print(f'Failed - {elem}')
        return None

    data = r.text if not raw else r.content
    return data

def save_html(path, data, raw=False):
    mode = 'w' if not raw else 'wb'
    with open(path, mode) as textfile:
        textfile.write(data)
        textfile.close()

## Download Layers

In [ ]:
layers = ["L1", "L23", "L4", "L5", "L6"]

In [ ]:
# !curl -o \
# /Users/antonel/Desktop/nmc-portal/widget/microcircuit/-/smw-facts/layer/{layer}.html \
# https://bbp.epfl.ch/nmc-portal/widget/microcircuit/-/smw-facts/layer/{layer}?p_p_state=exclusive

In [ ]:
path = 'nmc-portal/widget/microcircuit/-/smw-facts/layer'

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
origin = f'https://bbp.epfl.ch/{path}'
create_dir(destination)
for elem in layers:
    data = fetch_html(f"{origin}/{elem}?p_p_state=exclusive")
    save_html(f"{destination}/{elem}.html", data)
print('Done')

## Download MTypes

In [ ]:
# http://127.0.0.1:8082/widget/microcircuit/-/smw-facts/mtype/L5_BP?p_p_state=exclusive

In [ ]:
path = 'nmc-portal/widget/microcircuit/-/smw-facts/mtype'

In [ ]:
def find_mtypes():
    mtypes_file = open('assets/documents/10184/21773/nb_connection.json')
    mtypes_str = mtypes_file.read()
    founds = re.findall('"name":"(.+?)"', mtypes_str)
    return np.unique(founds)
founds = find_mtypes()
len(founds)

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
origin = f'https://bbp.epfl.ch/{path}'
create_dir(destination)
for elem in founds:
    url = f"{origin}/{elem}?p_p_state=exclusive"
    data = fetch_html(url)
    save_html(f"{destination}/{elem}.html", data)
print('Done')

## Download Pathways

In [ ]:
# http://127.0.0.1:8082/widget/microcircuit/-/smw-facts/pathway/L1_DAC-L1_HAC?p_p_state=exclusive

In [ ]:
path = 'nmc-portal/widget/microcircuit/-/smw-facts/pathway'

In [ ]:
def get_pathways():
    with open('assets/documents/10184/21773/pathways_list.json') as f:
        pathways1 = json.load(f)
    pathways1 = [s.replace(':L', '-L') for s in pathways1]
    
    with open('assets/documents/10184/21773/pathways_list_2.json') as f:
        pathways2 = json.load(f)
    pathways2 = [s.replace(':L', '-L') for s in pathways2]
    
    total = np.concatenate((pathways1, pathways2))
    total = np.unique(total)
    return total
founds = get_pathways()
len(founds)

#### As in the nmc portal L1_SLAC -> L1_SAC AND L1_DLAC -> L1_LAC


In [ ]:
changed_pathways = []
for pathway in founds:
    if 'L1_DLAC' in pathway:
        new_pathway = pathway.replace('L1_DLAC', 'L1_LAC')
        changed_pathways.append(new_pathway)
    if 'L1_SLAC' in pathway:
        new_pathway = pathway.replace('L1_SLAC', 'L1_SAC')
        changed_pathways.append(new_pathway)

changed_pathways.append('L1_SAC-L1_LAC')
changed_pathways.append('L1_LAC-L1_SAC')
founds = changed_pathways

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
origin = f'https://bbp.epfl.ch/{path}'
create_dir(destination)
for elem in founds:
    url = f"{origin}/{elem}?p_p_state=exclusive"
    d = Path(f"{destination}/{elem}.html")
    if d.exists():
        continue
    data = fetch_html(url)
    save_html(f"{destination}/{elem}.html", data)
print('Done')

In [ ]:
!ls ../{path} | grep html | wc -l

## Download Pathways Details pages

In [ ]:
# https://bbp.epfl.ch/nmc-portal/widget/microcircuit/-/smw-facts/pathway/L1_DAC-L1_DAC/details?p_p_state=exclusive

In [ ]:
path = 'nmc-portal/widget/microcircuit/-/smw-facts/pathway'
# founds = get_pathways()
founds = changed_pathways

destination = f'/Users/antonel/Desktop/{path}'
origin = f'https://bbp.epfl.ch/{path}'
create_dir(destination)
for elem in founds:
    d = Path(f"{destination}/{elem}/details/index.html")
    if d.exists():
        continue
    url = f"{origin}/{elem}/details?p_p_state=exclusive"
    data = fetch_html(url)
    destination_path = f"{destination}/{elem}/details"
    create_dir(destination_path)
    save_html(f"{destination_path}/index.html", data)
print('Done')

In [ ]:
!find ../{path} | grep details/index.html | wc -l

## Download Combos

In [ ]:
founds = [
#   combos gallery
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/color-base/color-base-min.js",
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/widget-base/assets/skins/sam/widget-base.css&/nmc-portal/html/js/aui/widget-modality/assets/skins/sam/widget-modality.css&/nmc-portal/html/js/aui/widget-stack/assets/skins/sam/widget-stack.css&/nmc-portal/html/js/aui/aui-image-viewer-base/assets/skins/sam/aui-image-viewer-base.css&/nmc-portal/html/js/aui/cssbutton/cssbutton-min.css&/nmc-portal/html/js/aui/aui-image-viewer-gallery/assets/skins/sam/aui-image-viewer-gallery.css",
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/anim-base/anim-base-min.js&/nmc-portal/html/js/aui/anim-color/anim-color-min.js&/nmc-portal/html/js/aui/anim-xy/anim-xy-min.js&/nmc-portal/html/js/aui/anim-curve/anim-curve-min.js&/nmc-portal/html/js/aui/anim-easing/anim-easing-min.js&/nmc-portal/html/js/aui/anim-node-plugin/anim-node-plugin-min.js&/nmc-portal/html/js/aui/anim-scroll/anim-scroll-min.js&/nmc-portal/html/js/aui/widget-modality/widget-modality-min.js&/nmc-portal/html/js/aui/widget-position/widget-position-min.js&/nmc-portal/html/js/aui/widget-position-align/widget-position-align-min.js&/nmc-portal/html/js/aui/widget-position-constrain/widget-position-constrain-min.js&/nmc-portal/html/js/aui/widget-stack/widget-stack-min.js&/nmc-portal/html/js/aui/widget-stdmod/widget-stdmod-min.js&/nmc-portal/html/js/aui/aui-event-delegate-change/aui-event-delegate-change-min.js",
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/aui-event-delegate-submit/aui-event-delegate-submit-min.js&/nmc-portal/html/js/aui/aui-image-viewer-base/aui-image-viewer-base-min.js&/nmc-portal/html/js/aui/aui-pagination/aui-pagination-min.js&/nmc-portal/html/js/aui/button-core/button-core-min.js&/nmc-portal/html/js/aui/button/button-min.js&/nmc-portal/html/js/aui/button-plugin/button-plugin-min.js&/nmc-portal/html/js/aui/button-group/button-group-min.js&/nmc-portal/html/js/aui/aui-button-core/aui-button-core-min.js&/nmc-portal/html/js/aui/aui-toolbar/aui-toolbar-min.js&/nmc-portal/html/js/aui/aui-image-viewer-gallery/aui-image-viewer-gallery-min.js&/nmc-portal/html/js/liferay/language.js",
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/widget-base/assets/skins/sam/widget-base.css&/nmc-portal/html/js/aui/widget-modality/assets/skins/sam/widget-modality.css&/nmc-portal/html/js/aui/widget-stack/assets/skins/sam/widget-stack.css&/nmc-portal/html/js/aui/aui-image-viewer-base/assets/skins/sam/aui-image-viewer-base.css&/nmc-portal/html/js/aui/cssbutton/cssbutton-min.css&/nmc-portal/html/js/aui/aui-image-viewer-gallery/assets/skins/sam/aui-image-viewer-gallery.css",
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/anim-base/anim-base-min.js&/nmc-portal/html/js/aui/anim-color/anim-color-min.js&/nmc-portal/html/js/aui/anim-xy/anim-xy-min.js&/nmc-portal/html/js/aui/anim-curve/anim-curve-min.js&/nmc-portal/html/js/aui/anim-easing/anim-easing-min.js&/nmc-portal/html/js/aui/anim-node-plugin/anim-node-plugin-min.js&/nmc-portal/html/js/aui/anim-scroll/anim-scroll-min.js&/nmc-portal/html/js/aui/widget-modality/widget-modality-min.js&/nmc-portal/html/js/aui/widget-position/widget-position-min.js&/nmc-portal/html/js/aui/widget-position-align/widget-position-align-min.js&/nmc-portal/html/js/aui/widget-position-constrain/widget-position-constrain-min.js&/nmc-portal/html/js/aui/widget-stack/widget-stack-min.js&/nmc-portal/html/js/aui/widget-stdmod/widget-stdmod-min.js&/nmc-portal/html/js/aui/aui-event-delegate-change/aui-event-delegate-change-min.js",
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/aui-event-delegate-submit/aui-event-delegate-submit-min.js&/nmc-portal/html/js/aui/aui-image-viewer-base/aui-image-viewer-base-min.js&/nmc-portal/html/js/aui/aui-pagination/aui-pagination-min.js&/nmc-portal/html/js/aui/button-core/button-core-min.js&/nmc-portal/html/js/aui/button/button-min.js&/nmc-portal/html/js/aui/button-plugin/button-plugin-min.js&/nmc-portal/html/js/aui/button-group/button-group-min.js&/nmc-portal/html/js/aui/aui-button-core/aui-button-core-min.js&/nmc-portal/html/js/aui/aui-toolbar/aui-toolbar-min.js&/nmc-portal/html/js/aui/aui-image-viewer-gallery/aui-image-viewer-gallery-min.js",
  
#   molecular-properties
#   "https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/color-base/color-base-min.js"
#   videos
"https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/color-base/color-base-min.js"

]

In [ ]:
subpath = 'videos'
path = f'nmc-portal/assets/combos/{subpath}'

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
create_dir(destination)

counter = 0
script_tags = []
css_tags = []

for elem in founds:
    url = elem
    data = fetch_html(url)

    if url.endswith('.css'):
        destination_path = Path(f'{destination}/{counter}.css')
        css_tags.append(f'<link href="./assets/combos/{subpath}/{counter}.css" rel="stylesheet" type="text/css">')
    if url.endswith('.js'):
        destination_path = Path(f'{destination}/{counter}.js')
        script_tags.append(f'<script src="./assets/combos/{subpath}/{counter}.js"></script>')

    counter = counter + 1

    save_html(destination_path, data)
print('Done')

In [ ]:
for css in css_tags:
    print(css)
for script in script_tags:
    print(script)

In [ ]:
<script src="./assets/html/combos/2.js"></script>
<script src="./assets/html/combos/3.js"></script>
<link href="./assets/html/combos/1.css" rel="stylesheet" type="text/css">
<link href="./assets/html/combos/2.css" rel="stylesheet" type="text/css">

## Download Assets

In [ ]:
founds = [
#   "https://bbp.epfl.ch/nmc-portal/html/js/aui/event-valuechange/event-valuechange-min.js",
#   "https://bbp.epfl.ch/nmc-portal/html/js/aui/event-tap/event-tap-min.js",
#   "https://bbp.epfl.ch/nmc-portal/html/js/aui/dd-ddm/dd-ddm-min.js",
#   "https://bbp.epfl.ch/nmc-portal/documents/10184/1204661/27_Blue Gene retro cavi.jpg",
#   "https://bbp.epfl.ch/nmc-portal/documents/10184/1204661/28_Blue Gene dettagli cavi retro.jpg",
#   "https://bbp.epfl.ch/nmc-portal/documents/10184/1204661/28_Blue Gene dettagli cavi retro.jpg",  
#   https://bbp.epfl.ch/nmc-portal/combo/?browserId=other&minifierType=&languageId=en_US&b=6200&t=1383330666000&/nmc-portal/html/js/aui/event-touch/event-touch-min.js&/nmc-portal/html/js/aui/event-move/event-move-min.js
]

In [ ]:
path = 'assets'

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
create_dir(destination)
raw = True
for elem in founds:
    url = elem
    data = fetch_html(url, raw)
    
    destination_path = Path(elem.replace('https://bbp.epfl.ch/nmc-portal', path))
    create_dir(destination_path.parent)

    save_html(destination_path, data, raw)
print('Done')

## Download Synaptome details page

In [ ]:
# https://bbp.epfl.ch/nmc-portal/widget/microcircuit/-/smw-facts/synaptome/L1_DAC/details?p_p_state=exclusive

In [ ]:
path = 'nmc-portal/widget/microcircuit/-/smw-facts/synaptome'

In [ ]:
founds = find_mtypes()
len(founds)

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
origin = f'https://bbp.epfl.ch/{path}'
create_dir(destination)
for elem in founds:
    url = f"{origin}/{elem}/details?p_p_state=exclusive"
    data = fetch_html(url)
    destination_path = f"{destination}/{elem}/details"
    create_dir(destination_path)
    save_html(f"{destination_path}/index.html", data)
print('Done')

In [ ]:
!ls ../{path} | wc -l

## Download MTypes details page

In [ ]:
# http://127.0.0.1:8082/widget/microcircuit/-/smw-facts/mtype/L1_DAC/details?p_p_state=exclusive

In [ ]:
path = 'nmc-portal/widget/microcircuit/-/smw-facts/mtype'

In [ ]:
founds = find_mtypes()
len(founds)

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
origin = f'https://bbp.epfl.ch/{path}'
create_dir(destination)
for elem in founds:
    url = f"{origin}/{elem}/details?p_p_state=exclusive"
    data = fetch_html(url)
    destination_path = f"{destination}/{elem}/details"
    create_dir(destination_path)
    save_html(f"{destination_path}/index.html", data)
print('Done')

## Download METypes details page

In [ ]:
# https://bbpteam.epfl.ch/public/nmc-portal-static/nmc-portal/widget/microcircuit/-/smw-facts/metype/L1_LAC_cNAC/details/index.html?p_p_state=exclusive

In [ ]:
path = 'nmc-portal/widget/microcircuit/-/smw-facts/metype'

In [ ]:
def find_metypes():
    with open('assets/documents/10184/21773/metypes.json') as f:
        metypes = json.load(f)
    return np.unique(metypes)

founds = find_metypes()
len(founds)

In [ ]:
destination = f'/Users/antonel/Desktop/{path}'
origin = f'https://bbp.epfl.ch/{path}'
create_dir(destination)
for elem in founds:
    url = f"{origin}/{elem}/details?p_p_state=exclusive"
    data = fetch_html(url)
    destination_path = f"{destination}/{elem}/details"
    create_dir(destination_path)
    save_html(f"{destination_path}/index.html", data)
print('Done')

## Change documents details links

In [ ]:
#/Users/antonel/Desktop/nmc-portal/widget/microcircuit/-/smw-facts/mtype/L1_DAC/details/index.html

In [ ]:
# documents_re = r'\"https://bbp\.epfl\.ch/nmc-portal/documents/.+?/.+?/(.+?)/.+?\"'
documents_re = r'[\'\"]https://bbp\.epfl\.ch/nmc-portal/documents/.+?/.+?/(.+?)/.+?[\'\"]'

In [6]:
def change_paths(documents_re, new_document_re, html):
    return re.sub(documents_re, new_document_re, html)

In [ ]:
# with open(path, 'r+') as f:
#     html = f.read()
#     new_html = change_paths(html)
#     f.seek(0)
#     f.write(new_html)
#     f.truncate()

In [ ]:
def change_document_paths(asset_type, new_asset_path):    
    base_folder = f'/Users/antonel/Desktop/nmc-portal/widget/microcircuit/-/smw-facts/{asset_type}'
    dir_item = os.listdir(base_folder)
    folders = [x for x in dir_item if os.path.isdir(base_folder + '/' + x) and x != '.ipynb_checkpoints']

    for folder in folders:
        path = f'{base_folder}/{folder}/details/index.html'
#         new_document_re = fr'"./assets/documents/static/{new_asset_path}/{folder}/\1"' # using ""
        new_document_re = fr"'./assets/documents/static/{new_asset_path}/{folder}/\1'" # using ''
        with open(path, 'r+') as f:
            html = f.read()
            new_html = change_paths(documents_re, new_document_re, html)
            f.seek(0)
            f.write(new_html)
            f.truncate()

In [ ]:
local_asset_type = 'mtype'
new_asset_path = 'MTypes'
change_document_paths(local_asset_type, new_asset_path)

In [ ]:
local_asset_type = 'pathway'
new_asset_path = 'Pathways'
change_document_paths(local_asset_type, new_asset_path)

In [ ]:
local_asset_type = 'synaptome'
new_asset_path = 'Synaptome'
change_document_paths(local_asset_type, new_asset_path)

In [ ]:
local_asset_type = 'metype'
new_asset_path = 'METypes'
change_document_paths(local_asset_type, new_asset_path)

## Change documents links 

In [ ]:
def change_file_link(local_asset_type, new_asset_path):
    base_folder = f'/Users/antonel/Desktop/nmc-portal/widget/microcircuit/-/smw-facts/{local_asset_type}'
    dir_item = os.listdir(base_folder)
    html_files = [x for x in dir_item if x.endswith('.html') and x != '.ipynb_checkpoints']

    for file in html_files:
        path = f'{base_folder}/{file}'
        folder_name = file.replace('.html', '')
        new_document_re = fr'"./assets/documents/static/{new_asset_path}/{folder_name}/\1"'
        with open(path, 'r+') as f:
            html = f.read()
            new_html = change_paths(documents_re, new_document_re, html)
            f.seek(0)
            f.write(new_html)
            f.truncate()

In [ ]:
local_asset_type = 'layer'
new_asset_path = 'CircuitLayers'
change_file_link(local_asset_type, new_asset_path)

## Generate zip to download page

In [ ]:
with open('assets/documents/10184/21773/download_tree.json') as f:
     tree = json.load(f)

In [ ]:
# tree

In [ ]:
def generate_command(variable):
    folder = variable.replace(':', '_')
#     origin = '/gpfs/bbp.cscs.ch/project/proj30/home/courcol/nmc-data/METypes'
    destination = '/gpfs/bbp.cscs.ch/project/proj30/home/courcol/nmc-data/downloads-zip'
    return f'echo {variable} && tar -cf {destination}/{variable}.zip ./{folder}*'

for layer in tree:
    cmd = generate_command(layer['text'])
    print(cmd)
    for node in layer['nodes']:
        cmd = generate_command(node['text'])
        print(cmd)
        for subnode in node['nodes']:
            cmd = generate_command(subnode['text'])
            print(cmd)

## Change combo script

In [ ]:
old_re = r'<script\n.+type="text\/javascript">var Liferay .+?<\/script>'
new_re = ' '

In [ ]:
base_folder = f'/Users/antonel/Desktop/nmc-portal'
dir_item = os.listdir(base_folder)
root_files = [x for x in dir_item if os.path.isfile(f'{base_folder}/{x}') and not x.startswith('.') and not x == 'microcircuit.html']

for file in root_files:
    print(file)
    path = f'{base_folder}/{file}'
    with open(path, 'r+') as f:
        html = f.read()
        new_html = change_paths(old_re, new_re, html)
        f.seek(0)
        f.write(new_html)
        f.truncate()

In [ ]:
# re.sub(documents_re, new_document_re, html)

In [ ]:
# <h3 id="title"> me-type fact sheet - L23_BP_bNAC</h3> <div id="content"> <div class="smw-details-view"> <ul id="tabs" class="nav nav-tabs" data-tabs="tabs"> <li title="C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33" class="smw-tab-selector active" data-tab="smw-tab-0"><a href="#">C230998A-I3_-_Scale_x1.00...</a></li> <li title="C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47" class="smw-tab-selector" data-tab="smw-tab-1"><a href="#">C230998A-I3_-_Scale_x1.00...</a></li> <li title="C230998A-I3_-_Clone_46" class="smw-tab-selector" data-tab="smw-tab-2"><a href="#">C230998A-I3_-_Clone_46</a></li> <li title="C230998A-I3_-_Clone_16" class="smw-tab-selector" data-tab="smw-tab-3"><a href="#">C230998A-I3_-_Clone_16</a></li> <li title="C230998A-I3_-_Clone_0" class="smw-tab-selector" data-tab="smw-tab-4"><a href="#">C230998A-I3_-_Clone_0</a></li> </ul> <div class="tab-content"> <div class="smw-tab-0 smw-active-tab smw-tab"> <div class="smw-panel-highlight"> <div class="row-fluid"> <div class="span3"> <h6 class="title-fact">Anatomy</h6> <dl class="smaller-fact"> <dt title='axon length'>axon length</dt> <dd class="factsheet-value">1,700 <span class="factsheet-unit">µm</span> <dt title='axon volume'>axon volume</dt> <dd class="factsheet-value">180 <span class="factsheet-unit">µm³</span> <dt title='axon maximum branch order'>axon maximum branch order</dt> <dd class="factsheet-value">10 <span class="factsheet-unit"></span> <dt title='axon maximum section length'>axon maximum section length</dt> <dd class="factsheet-value">330 <span class="factsheet-unit">µm</span> <dt title='basal volume'>basal volume</dt> <dd class="factsheet-value">280 <span class="factsheet-unit">µm³</span> <dt title='basal maximum branch order'>basal maximum branch order</dt> <dd class="factsheet-value">4 <span class="factsheet-unit"></span> <dt title='basal length'>basal length</dt> <dd class="factsheet-value">630 <span class="factsheet-unit">µm</span> <dt title='basal maximum section length'>basal maximum section length</dt> <dd class="factsheet-value">130 <span class="factsheet-unit">µm</span> <dt title='soma diameter'>soma diameter</dt> <dd class="factsheet-value">11 <span class="factsheet-unit">µm</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> <div> <span style="font-weight: bold;">Morphological data: </span><a href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Axon</a>,<a style="margin-left: 5px;" href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Dendrite</a> </div> </div> <!-- span3 --> <div class="span3"> <h6 class="title-fact">Physiology</h6> <dl class="smaller-fact"> <dt title=''>resting membrane potential</dt> <dd class="factsheet-value">-71.718 <span class="factsheet-unit">mV</span> <dt title=''>input resistance</dt> <dd class="factsheet-value">1,233.2 <span class="factsheet-unit">MOhm</span> <dt title=''>membrane time constant</dt> <dd class="factsheet-value">20.976 <span class="factsheet-unit">ms</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> </div> <!-- span3 --> <div class="span6"> <h6 class="title-fact">Channel mechanisms</h6> <dl class="channel-fact"> <dd> <table class="table"> <thead> <th style="width: 20%">sections</th> <th>mechanisms</th> </thead> <tr> <td style="width: 20%">dendrite</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.000229" data-latex="0.000229" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.000083" data-latex="0.000083" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000000" data-latex="0.000000" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000049*(-0.8696 + 2.087*exp(x*0.003))" data-latex="0.000049*(-0.8696 + 2.087*e^{x*0.003})" data-name="exponential">gIhbar</span> Ih</a> <a class="smw-equation" href="#"> <span data-equation="0.000022" data-latex="0.000022" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.001511" data-latex="0.001511" data-name="uniform">gK_Tstbar</span> K_Tst</a> </td> </tr> <tr> <td style="width: 20%">axonal</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.386953" data-latex="0.386953" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000400" data-latex="0.000400" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.001224" data-latex="0.001224" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.009017" data-latex="0.009017" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.001739" data-latex="0.001739" data-name="uniform">gamma</span> <span data-equation="468.069681" data-latex="468.069681" data-name="uniform">decay</span> CaDynamics_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.000554" data-latex="0.000554" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.001693" data-latex="0.001693" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.042115" data-latex="0.042115" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="3.999855" data-latex="3.999855" data-name="uniform">gNaTa_tbar</span> NaTa_t</a> </td> </tr> <tr> <td style="width: 20%">somatic</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.150747" data-latex="0.150747" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.503893" data-latex="0.503893" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000174" data-latex="0.000174" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.000523" data-latex="0.000523" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.003242" data-latex="0.003242" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000478" data-latex="0.000478" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.005446" data-latex="0.005446" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.039863" data-latex="0.039863" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="0.000500" data-latex="0.000500" data-name="uniform">gamma</span> <span data-equation="645.079741" data-latex="645.079741" data-name="uniform">decay</span> CaDynamics_E2</a> </td> </tr> </table> </dd> </dl> <div> <div style="float: right"> <a style="padding-left: 15px; padding-right: 15px; margin-top: 50px; margin-bottom: 10px;" class="btn btn-primary btn-block smw-detail-button" href="./assets/documents/static/METypes/L23_BP_bNAC/L23_BP_bNAC219_1.zip"> download model</a> </div> </div> <!-- download url --> </div> <!-- span9 --> </div> <!-- row-fluid--> </div> <!-- smw-panel-highlight --> <div class="row-fluid" style="margin-top: 20px;"> <div class="span3" style="text-align: center"> <div style="display: inline-block;"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33.png"><img class="picture" style="width: 100%; max-width: 400px;" src="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33.png"></a> </div> </div> <!-- span3 --> <div class="span6" style="text-align: center"> <div style="max-width: 600px; display: inline-block;"> <div id="morph" style="width: 100%; text-align: center; display: inline-block;" onresize="onResize"> <div id="img_morph"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33_xy_.png"> <img style="width: 100%; max-width: 600px;" class="picture" src="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33_xy_.png"> </a> </div> </div> </div> <div style="max-width: 600px; display: inline-block;" class="btn btn-primary btn-block smw-detail-button view3d" onclick="toggle_loadGeometry('./assets/documents/static/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33.bin', 'smw-tab-0')">view in 3D</div> </div> <!-- span 6 --> <div class="span3" style="text-align: center"> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33/epsp.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33/epsp.ogv" type="video/ogg"> "video tag not supported by your browser" </video> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33/bap.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y1.025_z1.000_-_Clone_33/bap.ogv" type="video/ogg"> </video> </div> </div> <!-- row-fluid--> <div class="row-fluid"> <div class="span3" style="text-align: center">response traces to step currents</div> <div class="span6" style="text-align: center"></div> <div class="span3" style="text-align: center">EPSP attenuation and bAP attenuation</div> </div> </div> <!-- smw-tab --> <div class="smw-tab-1 smw-tab"> <div class="smw-panel-highlight"> <div class="row-fluid"> <div class="span3"> <h6 class="title-fact">Anatomy</h6> <dl class="smaller-fact"> <dt title='axon length'>axon length</dt> <dd class="factsheet-value">1,600 <span class="factsheet-unit">µm</span> <dt title='axon volume'>axon volume</dt> <dd class="factsheet-value">150 <span class="factsheet-unit">µm³</span> <dt title='axon maximum branch order'>axon maximum branch order</dt> <dd class="factsheet-value">10 <span class="factsheet-unit"></span> <dt title='axon maximum section length'>axon maximum section length</dt> <dd class="factsheet-value">410 <span class="factsheet-unit">µm</span> <dt title='basal volume'>basal volume</dt> <dd class="factsheet-value">260 <span class="factsheet-unit">µm³</span> <dt title='basal maximum branch order'>basal maximum branch order</dt> <dd class="factsheet-value">4 <span class="factsheet-unit"></span> <dt title='basal length'>basal length</dt> <dd class="factsheet-value">550 <span class="factsheet-unit">µm</span> <dt title='basal maximum section length'>basal maximum section length</dt> <dd class="factsheet-value">130 <span class="factsheet-unit">µm</span> <dt title='soma diameter'>soma diameter</dt> <dd class="factsheet-value">11 <span class="factsheet-unit">µm</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> <div> <span style="font-weight: bold;">Morphological data: </span><a href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Axon</a>,<a style="margin-left: 5px;" href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Dendrite</a> </div> </div> <!-- span3 --> <div class="span3"> <h6 class="title-fact">Physiology</h6> <dl class="smaller-fact"> <dt title=''>resting membrane potential</dt> <dd class="factsheet-value">-71.845 <span class="factsheet-unit">mV</span> <dt title=''>input resistance</dt> <dd class="factsheet-value">1,281.8 <span class="factsheet-unit">MOhm</span> <dt title=''>membrane time constant</dt> <dd class="factsheet-value">20.039 <span class="factsheet-unit">ms</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> </div> <!-- span3 --> <div class="span6"> <h6 class="title-fact">Channel mechanisms</h6> <dl class="channel-fact"> <dd> <table class="table"> <thead> <th style="width: 20%">sections</th> <th>mechanisms</th> </thead> <tr> <td style="width: 20%">dendrite</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.000229" data-latex="0.000229" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.000083" data-latex="0.000083" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000000" data-latex="0.000000" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000049*(-0.8696 + 2.087*exp(x*0.003))" data-latex="0.000049*(-0.8696 + 2.087*e^{x*0.003})" data-name="exponential">gIhbar</span> Ih</a> <a class="smw-equation" href="#"> <span data-equation="0.000022" data-latex="0.000022" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.001511" data-latex="0.001511" data-name="uniform">gK_Tstbar</span> K_Tst</a> </td> </tr> <tr> <td style="width: 20%">axonal</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.386953" data-latex="0.386953" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000400" data-latex="0.000400" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.001224" data-latex="0.001224" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.009017" data-latex="0.009017" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.001739" data-latex="0.001739" data-name="uniform">gamma</span> <span data-equation="468.069681" data-latex="468.069681" data-name="uniform">decay</span> CaDynamics_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.000554" data-latex="0.000554" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.001693" data-latex="0.001693" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.042115" data-latex="0.042115" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="3.999855" data-latex="3.999855" data-name="uniform">gNaTa_tbar</span> NaTa_t</a> </td> </tr> <tr> <td style="width: 20%">somatic</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.150747" data-latex="0.150747" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.503893" data-latex="0.503893" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000174" data-latex="0.000174" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.000523" data-latex="0.000523" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.003242" data-latex="0.003242" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000478" data-latex="0.000478" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.005446" data-latex="0.005446" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.039863" data-latex="0.039863" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="0.000500" data-latex="0.000500" data-name="uniform">gamma</span> <span data-equation="645.079741" data-latex="645.079741" data-name="uniform">decay</span> CaDynamics_E2</a> </td> </tr> </table> </dd> </dl> <div> <div style="float: right"> <a style="padding-left: 15px; padding-right: 15px; margin-top: 50px; margin-bottom: 10px;" class="btn btn-primary btn-block smw-detail-button" href="./assets/documents/static/METypes/L23_BP_bNAC/L23_BP_bNAC219_2.zip"> download model</a> </div> </div> <!-- download url --> </div> <!-- span9 --> </div> <!-- row-fluid--> </div> <!-- smw-panel-highlight --> <div class="row-fluid" style="margin-top: 20px;"> <div class="span3" style="text-align: center"> <div style="display: inline-block;"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47.png"><img class="picture" style="width: 100%; max-width: 400px;" src="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47.png"></a> </div> </div> <!-- span3 --> <div class="span6" style="text-align: center"> <div style="max-width: 600px; display: inline-block;"> <div id="morph" style="width: 100%; text-align: center; display: inline-block;" onresize="onResize"> <div id="img_morph"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47_xy_.png"> <img style="width: 100%; max-width: 600px;" class="picture" src="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47_xy_.png"> </a> </div> </div> </div> <div style="max-width: 600px; display: inline-block;" class="btn btn-primary btn-block smw-detail-button view3d" onclick="toggle_loadGeometry('./assets/documents/static/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47.bin', 'smw-tab-1')">view in 3D</div> </div> <!-- span 6 --> <div class="span3" style="text-align: center"> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47/epsp.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47/epsp.ogv" type="video/ogg"> "video tag not supported by your browser" </video> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47/bap.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Scale_x1.000_y0.950_z1.000_-_Clone_47/bap.ogv" type="video/ogg"> </video> </div> </div> <!-- row-fluid--> <div class="row-fluid"> <div class="span3" style="text-align: center">response traces to step currents</div> <div class="span6" style="text-align: center"></div> <div class="span3" style="text-align: center">EPSP attenuation and bAP attenuation</div> </div> </div> <!-- smw-tab --> <div class="smw-tab-2 smw-tab"> <div class="smw-panel-highlight"> <div class="row-fluid"> <div class="span3"> <h6 class="title-fact">Anatomy</h6> <dl class="smaller-fact"> <dt title='axon length'>axon length</dt> <dd class="factsheet-value">1,700 <span class="factsheet-unit">µm</span> <dt title='axon volume'>axon volume</dt> <dd class="factsheet-value">160 <span class="factsheet-unit">µm³</span> <dt title='axon maximum branch order'>axon maximum branch order</dt> <dd class="factsheet-value">10 <span class="factsheet-unit"></span> <dt title='axon maximum section length'>axon maximum section length</dt> <dd class="factsheet-value">440 <span class="factsheet-unit">µm</span> <dt title='basal volume'>basal volume</dt> <dd class="factsheet-value">270 <span class="factsheet-unit">µm³</span> <dt title='basal maximum branch order'>basal maximum branch order</dt> <dd class="factsheet-value">4 <span class="factsheet-unit"></span> <dt title='basal length'>basal length</dt> <dd class="factsheet-value">580 <span class="factsheet-unit">µm</span> <dt title='basal maximum section length'>basal maximum section length</dt> <dd class="factsheet-value">120 <span class="factsheet-unit">µm</span> <dt title='soma diameter'>soma diameter</dt> <dd class="factsheet-value">11 <span class="factsheet-unit">µm</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> <div> <span style="font-weight: bold;">Morphological data: </span><a href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Axon</a>,<a style="margin-left: 5px;" href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Dendrite</a> </div> </div> <!-- span3 --> <div class="span3"> <h6 class="title-fact">Physiology</h6> <dl class="smaller-fact"> <dt title=''>resting membrane potential</dt> <dd class="factsheet-value">-71.79 <span class="factsheet-unit">mV</span> <dt title=''>input resistance</dt> <dd class="factsheet-value">1,259.9 <span class="factsheet-unit">MOhm</span> <dt title=''>membrane time constant</dt> <dd class="factsheet-value">20.377 <span class="factsheet-unit">ms</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> </div> <!-- span3 --> <div class="span6"> <h6 class="title-fact">Channel mechanisms</h6> <dl class="channel-fact"> <dd> <table class="table"> <thead> <th style="width: 20%">sections</th> <th>mechanisms</th> </thead> <tr> <td style="width: 20%">dendrite</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.000229" data-latex="0.000229" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.000083" data-latex="0.000083" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000000" data-latex="0.000000" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000049*(-0.8696 + 2.087*exp(x*0.003))" data-latex="0.000049*(-0.8696 + 2.087*e^{x*0.003})" data-name="exponential">gIhbar</span> Ih</a> <a class="smw-equation" href="#"> <span data-equation="0.000022" data-latex="0.000022" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.001511" data-latex="0.001511" data-name="uniform">gK_Tstbar</span> K_Tst</a> </td> </tr> <tr> <td style="width: 20%">axonal</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.386953" data-latex="0.386953" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000400" data-latex="0.000400" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.001224" data-latex="0.001224" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.009017" data-latex="0.009017" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.001739" data-latex="0.001739" data-name="uniform">gamma</span> <span data-equation="468.069681" data-latex="468.069681" data-name="uniform">decay</span> CaDynamics_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.000554" data-latex="0.000554" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.001693" data-latex="0.001693" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.042115" data-latex="0.042115" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="3.999855" data-latex="3.999855" data-name="uniform">gNaTa_tbar</span> NaTa_t</a> </td> </tr> <tr> <td style="width: 20%">somatic</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.150747" data-latex="0.150747" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.503893" data-latex="0.503893" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000174" data-latex="0.000174" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.000523" data-latex="0.000523" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.003242" data-latex="0.003242" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000478" data-latex="0.000478" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.005446" data-latex="0.005446" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.039863" data-latex="0.039863" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="0.000500" data-latex="0.000500" data-name="uniform">gamma</span> <span data-equation="645.079741" data-latex="645.079741" data-name="uniform">decay</span> CaDynamics_E2</a> </td> </tr> </table> </dd> </dl> <div> <div style="float: right"> <a style="padding-left: 15px; padding-right: 15px; margin-top: 50px; margin-bottom: 10px;" class="btn btn-primary btn-block smw-detail-button" href="./assets/documents/static/METypes/L23_BP_bNAC/L23_BP_bNAC219_3.zip"> download model</a> </div> </div> <!-- download url --> </div> <!-- span9 --> </div> <!-- row-fluid--> </div> <!-- smw-panel-highlight --> <div class="row-fluid" style="margin-top: 20px;"> <div class="span3" style="text-align: center"> <div style="display: inline-block;"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Clone_46.png"><img class="picture" style="width: 100%; max-width: 400px;" src="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Clone_46.png"></a> </div> </div> <!-- span3 --> <div class="span6" style="text-align: center"> <div style="max-width: 600px; display: inline-block;"> <div id="morph" style="width: 100%; text-align: center; display: inline-block;" onresize="onResize"> <div id="img_morph"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Clone_46_xy_.png"> <img style="width: 100%; max-width: 600px;" class="picture" src="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Clone_46_xy_.png"> </a> </div> </div> </div> <div style="max-width: 600px; display: inline-block;" class="btn btn-primary btn-block smw-detail-button view3d" onclick="toggle_loadGeometry('./assets/documents/static/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_46.bin', 'smw-tab-2')">view in 3D</div> </div> <!-- span 6 --> <div class="span3" style="text-align: center"> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_46/epsp.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_46/epsp.ogv" type="video/ogg"> "video tag not supported by your browser" </video> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_46/bap.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_46/bap.ogv" type="video/ogg"> </video> </div> </div> <!-- row-fluid--> <div class="row-fluid"> <div class="span3" style="text-align: center">response traces to step currents</div> <div class="span6" style="text-align: center"></div> <div class="span3" style="text-align: center">EPSP attenuation and bAP attenuation</div> </div> </div> <!-- smw-tab --> <div class="smw-tab-3 smw-tab"> <div class="smw-panel-highlight"> <div class="row-fluid"> <div class="span3"> <h6 class="title-fact">Anatomy</h6> <dl class="smaller-fact"> <dt title='axon length'>axon length</dt> <dd class="factsheet-value">1,700 <span class="factsheet-unit">µm</span> <dt title='axon volume'>axon volume</dt> <dd class="factsheet-value">170 <span class="factsheet-unit">µm³</span> <dt title='axon maximum branch order'>axon maximum branch order</dt> <dd class="factsheet-value">10 <span class="factsheet-unit"></span> <dt title='axon maximum section length'>axon maximum section length</dt> <dd class="factsheet-value">480 <span class="factsheet-unit">µm</span> <dt title='basal volume'>basal volume</dt> <dd class="factsheet-value">280 <span class="factsheet-unit">µm³</span> <dt title='basal maximum branch order'>basal maximum branch order</dt> <dd class="factsheet-value">4 <span class="factsheet-unit"></span> <dt title='basal length'>basal length</dt> <dd class="factsheet-value">620 <span class="factsheet-unit">µm</span> <dt title='basal maximum section length'>basal maximum section length</dt> <dd class="factsheet-value">120 <span class="factsheet-unit">µm</span> <dt title='soma diameter'>soma diameter</dt> <dd class="factsheet-value">11 <span class="factsheet-unit">µm</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> <div> <span style="font-weight: bold;">Morphological data: </span><a href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Axon</a>,<a style="margin-left: 5px;" href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Dendrite</a> </div> </div> <!-- span3 --> <div class="span3"> <h6 class="title-fact">Physiology</h6> <dl class="smaller-fact"> <dt title=''>resting membrane potential</dt> <dd class="factsheet-value">-71.737 <span class="factsheet-unit">mV</span> <dt title=''>input resistance</dt> <dd class="factsheet-value">1,240.3 <span class="factsheet-unit">MOhm</span> <dt title=''>membrane time constant</dt> <dd class="factsheet-value">20.814 <span class="factsheet-unit">ms</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> </div> <!-- span3 --> <div class="span6"> <h6 class="title-fact">Channel mechanisms</h6> <dl class="channel-fact"> <dd> <table class="table"> <thead> <th style="width: 20%">sections</th> <th>mechanisms</th> </thead> <tr> <td style="width: 20%">dendrite</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.000229" data-latex="0.000229" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.000083" data-latex="0.000083" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000000" data-latex="0.000000" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000049*(-0.8696 + 2.087*exp(x*0.003))" data-latex="0.000049*(-0.8696 + 2.087*e^{x*0.003})" data-name="exponential">gIhbar</span> Ih</a> <a class="smw-equation" href="#"> <span data-equation="0.000022" data-latex="0.000022" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.001511" data-latex="0.001511" data-name="uniform">gK_Tstbar</span> K_Tst</a> </td> </tr> <tr> <td style="width: 20%">axonal</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.386953" data-latex="0.386953" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000400" data-latex="0.000400" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.001224" data-latex="0.001224" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.009017" data-latex="0.009017" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.001739" data-latex="0.001739" data-name="uniform">gamma</span> <span data-equation="468.069681" data-latex="468.069681" data-name="uniform">decay</span> CaDynamics_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.000554" data-latex="0.000554" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.001693" data-latex="0.001693" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.042115" data-latex="0.042115" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="3.999855" data-latex="3.999855" data-name="uniform">gNaTa_tbar</span> NaTa_t</a> </td> </tr> <tr> <td style="width: 20%">somatic</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.150747" data-latex="0.150747" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.503893" data-latex="0.503893" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000174" data-latex="0.000174" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.000523" data-latex="0.000523" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.003242" data-latex="0.003242" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000478" data-latex="0.000478" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.005446" data-latex="0.005446" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.039863" data-latex="0.039863" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="0.000500" data-latex="0.000500" data-name="uniform">gamma</span> <span data-equation="645.079741" data-latex="645.079741" data-name="uniform">decay</span> CaDynamics_E2</a> </td> </tr> </table> </dd> </dl> <div> <div style="float: right"> <a style="padding-left: 15px; padding-right: 15px; margin-top: 50px; margin-bottom: 10px;" class="btn btn-primary btn-block smw-detail-button" href="./assets/documents/static/METypes/L23_BP_bNAC/L23_BP_bNAC219_4.zip"> download model</a> </div> </div> <!-- download url --> </div> <!-- span9 --> </div> <!-- row-fluid--> </div> <!-- smw-panel-highlight --> <div class="row-fluid" style="margin-top: 20px;"> <div class="span3" style="text-align: center"> <div style="display: inline-block;"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Clone_16.png"><img class="picture" style="width: 100%; max-width: 400px;" src="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Clone_16.png"></a> </div> </div> <!-- span3 --> <div class="span6" style="text-align: center"> <div style="max-width: 600px; display: inline-block;"> <div id="morph" style="width: 100%; text-align: center; display: inline-block;" onresize="onResize"> <div id="img_morph"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Clone_16_xy_.png"> <img style="width: 100%; max-width: 600px;" class="picture" src="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Clone_16_xy_.png"> </a> </div> </div> </div> <div style="max-width: 600px; display: inline-block;" class="btn btn-primary btn-block smw-detail-button view3d" onclick="toggle_loadGeometry('./assets/documents/static/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_16.bin', 'smw-tab-3')">view in 3D</div> </div> <!-- span 6 --> <div class="span3" style="text-align: center"> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_16/epsp.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_16/epsp.ogv" type="video/ogg"> "video tag not supported by your browser" </video> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_16/bap.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_16/bap.ogv" type="video/ogg"> </video> </div> </div> <!-- row-fluid--> <div class="row-fluid"> <div class="span3" style="text-align: center">response traces to step currents</div> <div class="span6" style="text-align: center"></div> <div class="span3" style="text-align: center">EPSP attenuation and bAP attenuation</div> </div> </div> <!-- smw-tab --> <div class="smw-tab-4 smw-tab"> <div class="smw-panel-highlight"> <div class="row-fluid"> <div class="span3"> <h6 class="title-fact">Anatomy</h6> <dl class="smaller-fact"> <dt title='axon length'>axon length</dt> <dd class="factsheet-value">1,800 <span class="factsheet-unit">µm</span> <dt title='axon volume'>axon volume</dt> <dd class="factsheet-value">170 <span class="factsheet-unit">µm³</span> <dt title='axon maximum branch order'>axon maximum branch order</dt> <dd class="factsheet-value">10 <span class="factsheet-unit"></span> <dt title='axon maximum section length'>axon maximum section length</dt> <dd class="factsheet-value">490 <span class="factsheet-unit">µm</span> <dt title='basal volume'>basal volume</dt> <dd class="factsheet-value">260 <span class="factsheet-unit">µm³</span> <dt title='basal maximum branch order'>basal maximum branch order</dt> <dd class="factsheet-value">4 <span class="factsheet-unit"></span> <dt title='basal length'>basal length</dt> <dd class="factsheet-value">540 <span class="factsheet-unit">µm</span> <dt title='basal maximum section length'>basal maximum section length</dt> <dd class="factsheet-value">81 <span class="factsheet-unit">µm</span> <dt title='soma diameter'>soma diameter</dt> <dd class="factsheet-value">11 <span class="factsheet-unit">µm</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> <div> <span style="font-weight: bold;">Morphological data: </span><a href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Axon</a>,<a style="margin-left: 5px;" href="http://microcircuits.epfl.ch/#/animal/e513973d-b806-11e4-996a-6003088da632">Dendrite</a> </div> </div> <!-- span3 --> <div class="span3"> <h6 class="title-fact">Physiology</h6> <dl class="smaller-fact"> <dt title=''>resting membrane potential</dt> <dd class="factsheet-value">-71.85 <span class="factsheet-unit">mV</span> <dt title=''>input resistance</dt> <dd class="factsheet-value">1,283.7 <span class="factsheet-unit">MOhm</span> <dt title=''>membrane time constant</dt> <dd class="factsheet-value">19.829 <span class="factsheet-unit">ms</span> <!-- for valueItem --> </dl> <!--dl-horizontal--> </div> <!-- span3 --> <div class="span6"> <h6 class="title-fact">Channel mechanisms</h6> <dl class="channel-fact"> <dd> <table class="table"> <thead> <th style="width: 20%">sections</th> <th>mechanisms</th> </thead> <tr> <td style="width: 20%">dendrite</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.000229" data-latex="0.000229" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.000083" data-latex="0.000083" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000000" data-latex="0.000000" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000049*(-0.8696 + 2.087*exp(x*0.003))" data-latex="0.000049*(-0.8696 + 2.087*e^{x*0.003})" data-name="exponential">gIhbar</span> Ih</a> <a class="smw-equation" href="#"> <span data-equation="0.000022" data-latex="0.000022" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.001511" data-latex="0.001511" data-name="uniform">gK_Tstbar</span> K_Tst</a> </td> </tr> <tr> <td style="width: 20%">axonal</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.386953" data-latex="0.386953" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000400" data-latex="0.000400" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.001224" data-latex="0.001224" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.009017" data-latex="0.009017" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.001739" data-latex="0.001739" data-name="uniform">gamma</span> <span data-equation="468.069681" data-latex="468.069681" data-name="uniform">decay</span> CaDynamics_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.000554" data-latex="0.000554" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.001693" data-latex="0.001693" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.042115" data-latex="0.042115" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="3.999855" data-latex="3.999855" data-name="uniform">gNaTa_tbar</span> NaTa_t</a> </td> </tr> <tr> <td style="width: 20%">somatic</td> <td> <a class="smw-equation" href="#"> <span data-equation="0.150747" data-latex="0.150747" data-name="uniform">gNaTs2_tbar</span> NaTs2_t</a> <a class="smw-equation" href="#"> <span data-equation="0.503893" data-latex="0.503893" data-name="uniform">gSKv3_1bar</span> SKv3_1</a> <a class="smw-equation" href="#"> <span data-equation="0.000174" data-latex="0.000174" data-name="uniform">gCabar</span> Ca</a> <a class="smw-equation" href="#"> <span data-equation="0.000523" data-latex="0.000523" data-name="uniform">gSK_E2bar</span> SK_E2</a> <a class="smw-equation" href="#"> <span data-equation="0.003242" data-latex="0.003242" data-name="uniform">gCa_LVAstbar</span> Ca_LVAst</a> <a class="smw-equation" href="#"> <span data-equation="0.000001" data-latex="0.000001" data-name="uniform">gNap_Et2bar</span> Nap_Et2</a> <a class="smw-equation" href="#"> <span data-equation="0.000478" data-latex="0.000478" data-name="uniform">gImbar</span> Im</a> <a class="smw-equation" href="#"> <span data-equation="0.005446" data-latex="0.005446" data-name="uniform">gK_Pstbar</span> K_Pst</a> <a class="smw-equation" href="#"> <span data-equation="0.039863" data-latex="0.039863" data-name="uniform">gK_Tstbar</span> K_Tst</a> <a class="smw-equation" href="#"> <span data-equation="0.000500" data-latex="0.000500" data-name="uniform">gamma</span> <span data-equation="645.079741" data-latex="645.079741" data-name="uniform">decay</span> CaDynamics_E2</a> </td> </tr> </table> </dd> </dl> <div> <div style="float: right"> <a style="padding-left: 15px; padding-right: 15px; margin-top: 50px; margin-bottom: 10px;" class="btn btn-primary btn-block smw-detail-button" href="./assets/documents/static/METypes/L23_BP_bNAC/L23_BP_bNAC219_5.zip"> download model</a> </div> </div> <!-- download url --> </div> <!-- span9 --> </div> <!-- row-fluid--> </div> <!-- smw-panel-highlight --> <div class="row-fluid" style="margin-top: 20px;"> <div class="span3" style="text-align: center"> <div style="display: inline-block;"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Clone_0.png"><img class="picture" style="width: 100%; max-width: 400px;" src="./assets/documents/static/METypes/L23_BP_bNAC/bNAC_C230998A-I3_-_Clone_0.png"></a> </div> </div> <!-- span3 --> <div class="span6" style="text-align: center"> <div style="max-width: 600px; display: inline-block;"> <div id="morph" style="width: 100%; text-align: center; display: inline-block;" onresize="onResize"> <div id="img_morph"> <a class="smw-simple-viewer" href="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Clone_0_xy_.png"> <img style="width: 100%; max-width: 600px;" class="picture" src="./assets/documents/static/METypes/L23_BP_bNAC/_C230998A-I3_-_Clone_0_xy_.png"> </a> </div> </div> </div> <div style="max-width: 600px; display: inline-block;" class="btn btn-primary btn-block smw-detail-button view3d" onclick="toggle_loadGeometry('./assets/documents/static/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_0.bin', 'smw-tab-4')">view in 3D</div> </div> <!-- span 6 --> <div class="span3" style="text-align: center"> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_0/epsp.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_0/epsp.ogv" type="video/ogg"> "video tag not supported by your browser" </video> <video loop controls preload="none" width="320" height="240"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_0/bap.mp4" type="video/mp4"> <source src="https://bbp.epfl.ch/project/media/nmc-portal/METypes/L23_BP_bNAC/C230998A-I3_-_Clone_0/bap.ogv" type="video/ogg"> </video> </div> </div> <!-- row-fluid--> <div class="row-fluid"> <div class="span3" style="text-align: center">response traces to step currents</div> <div class="span6" style="text-align: center"></div> <div class="span3" style="text-align: center">EPSP attenuation and bAP attenuation</div> </div> </div> <!-- smw-tab --> </div> <!-- tab-content --> </div> <!-- smw-details-view --> <div class="accordion-group lfr-panel lfr-panel-extended" id="comments-panel"> <div class="accordion-heading toggler-header toggler-header-collapsed" data-persist-id=""> <div class="accordion-toggle"> <span class="title-text"> Comments (0) </span> </div> </div> <div class="toggler-content toggler-content-collapsed" id="comments-panelContent"> <div class="accordion-inner"> <div class="hide lfr-message-response" id="_smwfactsportlet_WAR_smwfactsportlet_discussion-status-messages"></div> <div class="taglib-discussion"> <form action="https://bbp.epfl.ch/nmc-portal/microcircuit?p_p_id=smwfactsportlet_WAR_smwfactsportlet&amp;p_p_lifecycle=1&amp;p_p_state=exclusive&amp;p_p_mode=view&amp;_smwfactsportlet_WAR_smwfactsportlet_javax.portlet.action=invokeTaglibDiscussion" class="form " id="_smwfactsportlet_WAR_smwfactsportlet_fm1" method="post" name="_smwfactsportlet_WAR_smwfactsportlet_fm1" > <input name="_smwfactsportlet_WAR_smwfactsportlet_formDate" type="hidden" value="1610979783399" /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_randomNamespace"    name="_smwfactsportlet_WAR_smwfactsportlet_randomNamespace"     type="hidden" value="raub_"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_raub_cmd"    name="_smwfactsportlet_WAR_smwfactsportlet_cmd"     type="hidden" value=""   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_redirect"    name="_smwfactsportlet_WAR_smwfactsportlet_redirect"     type="hidden" value="&#x2f;nmc-portal&#x2f;widget&#x2f;microcircuit&#x2f;-&#x2f;smw-facts&#x2f;metype&#x2f;L23_BP_bNAC&#x2f;details&#x3f;p_p_state&#x3d;exclusive"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_contentURL"    name="_smwfactsportlet_WAR_smwfactsportlet_contentURL"     type="hidden" value="https&#x3a;&#x2f;&#x2f;bbp&#x2e;epfl&#x2e;ch&#x2f;nmc-portal&#x2f;microcircuit&#x2f;-&#x2f;smw-facts&#x2f;metype&#x2f;L23_BP_bNAC&#x2f;details&#x3f;p_p_state&#x3d;exclusive"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_assetEntryVisible"    name="_smwfactsportlet_WAR_smwfactsportlet_assetEntryVisible"     type="hidden" value="true"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_className"    name="_smwfactsportlet_WAR_smwfactsportlet_className"     type="hidden" value="com&#x2e;liferay&#x2e;portlet&#x2e;documentlibrary&#x2e;model&#x2e;DLFolder"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_classPK"    name="_smwfactsportlet_WAR_smwfactsportlet_classPK"     type="hidden" value="51959"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_permissionClassName"    name="_smwfactsportlet_WAR_smwfactsportlet_permissionClassName"     type="hidden" value="com&#x2e;liferay&#x2e;portlet&#x2e;documentlibrary&#x2e;model&#x2e;DLFolder"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_permissionClassPK"    name="_smwfactsportlet_WAR_smwfactsportlet_permissionClassPK"     type="hidden" value="51959"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_permissionOwnerId"    name="_smwfactsportlet_WAR_smwfactsportlet_permissionOwnerId"     type="hidden" value="12662"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_messageId"    name="_smwfactsportlet_WAR_smwfactsportlet_messageId"     type="hidden" value=""   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_threadId"    name="_smwfactsportlet_WAR_smwfactsportlet_threadId"     type="hidden" value="11268423"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_parentMessageId"    name="_smwfactsportlet_WAR_smwfactsportlet_parentMessageId"     type="hidden" value=""   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_body"    name="_smwfactsportlet_WAR_smwfactsportlet_body"     type="hidden" value=""   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_workflowAction"    name="_smwfactsportlet_WAR_smwfactsportlet_workflowAction"     type="hidden" value="1"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_ajax"    name="_smwfactsportlet_WAR_smwfactsportlet_ajax"     type="hidden" value="true"   /> <fieldset class="fieldset add-comment" id="_smwfactsportlet_WAR_smwfactsportlet_raub_messageScroll0" > <div class=""> <div id="raub_messageScroll11268422"> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_messageId0"    name="_smwfactsportlet_WAR_smwfactsportlet_messageId0"     type="hidden" value="11268422"   /> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_parentMessageId0"    name="_smwfactsportlet_WAR_smwfactsportlet_parentMessageId0"     type="hidden" value="11268422"   /> </div> No comments yet. <a href="javascript:raub_showForm('raub_postReplyForm0', '_smwfactsportlet_WAR_smwfactsportlet_raub_postReplyBody0');">Be the first.</a> <input class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_emailAddress"    name="_smwfactsportlet_WAR_smwfactsportlet_emailAddress"     type="hidden" value=""   /> <div id="raub_postReplyForm0" style="display: none;"> <div class="control-group"> <label class="control-label" for="_smwfactsportlet_WAR_smwfactsportlet_raub_postReplyBody0"> Comment </label> <textarea class="field"  id="_smwfactsportlet_WAR_smwfactsportlet_raub_postReplyBody0"  name="_smwfactsportlet_WAR_smwfactsportlet_postReplyBody0"       wrap="soft" style="height: 100px; max-width: 500px;" ></textarea> </label> </div> <div class="button-holder " > <button class="btn btn-comment" onClick="raub_postReply(0);" type='button' id="_smwfactsportlet_WAR_smwfactsportlet_raub_postReplyButton0" > Submit for Publication </button> <button class="btn btn-comment btn-cancel" onClick="document.getElementById('raub_postReplyForm0').style.display = 'none'; document.getElementById('_smwfactsportlet_WAR_smwfactsportlet_raub_postReplyBody0').value = ''; void('');" type='button' > Cancel </button> </div> </div> </div></fieldset> </form> <script type="text/javascript">AUI().use("liferay-form",function(a){(function(){Liferay.Form.register({id:"_smwfactsportlet_WAR_smwfactsportlet_fm1",fieldRules:[]})})()});</script> </div> <script type="text/javascript">function raub_hideForm(c,b,a){document.getElementById(c).style.display="none";document.getElementById(b).value=a}function raub_scrollIntoView(a){document.getElementById("raub_messageScroll"+a).scrollIntoView()}function raub_showForm(b,a){document.getElementById(b).style.display="block";document.getElementById(a).focus()}Liferay.provide(window,"raub_afterLogin",function(d,b){var a=AUI();var c=a.one("#_smwfactsportlet_WAR_smwfactsportlet_fm1");c.one("#_smwfactsportlet_WAR_smwfactsportlet_emailAddress").val(d);_smwfactsportlet_WAR_smwfactsportlet_sendMessage(c,!b);location.hash+="#_message_"},["aui-base"]);Liferay.provide(window,"raub_deleteMessage",function(c){var a=AUI();var d=a.one("#_smwfactsportlet_WAR_smwfactsportlet_fm1");var b=d.one("#_smwfactsportlet_WAR_smwfactsportlet_messageId"+c).val();d.one("#_smwfactsportlet_WAR_smwfactsportlet_raub_cmd").val("delete");d.one("#_smwfactsportlet_WAR_smwfactsportlet_messageId").val(b);_smwfactsportlet_WAR_smwfactsportlet_sendMessage(d)},["aui-base"]);Liferay.provide(window,"_smwfactsportlet_WAR_smwfactsportlet_onMessagePosted",function(a,b){Liferay.after("smwfactsportlet_WAR_smwfactsportlet:portletRefreshed",function(d){var c=AUI();_smwfactsportlet_WAR_smwfactsportlet_showStatusMessage("success","\u0059\u006f\u0075\u0072\u0020\u0072\u0065\u0071\u0075\u0065\u0073\u0074\u0020\u0070\u0072\u006f\u0063\u0065\u0073\u0073\u0065\u0064\u0020\u0073\u0075\u0063\u0063\u0065\u0073\u0073\u0066\u0075\u006c\u006c\u0079\u002e");location.hash="#"+c.one("#_smwfactsportlet_WAR_smwfactsportlet_randomNamespace").val()+"message_"+a.messageId});if(b){window.location.reload()}else{Liferay.Portlet.refresh("#p_p_id_smwfactsportlet_WAR_smwfactsportlet_")}},["aui-base"]);Liferay.provide(window,"raub_postReply",function(c){var b=AUI();var d=b.one("#_smwfactsportlet_WAR_smwfactsportlet_fm1");var a=d.one("#_smwfactsportlet_WAR_smwfactsportlet_raub_postReplyBody"+c).val();var e=d.one("#_smwfactsportlet_WAR_smwfactsportlet_parentMessageId"+c).val();d.one("#_smwfactsportlet_WAR_smwfactsportlet_raub_cmd").val("add");d.one("#_smwfactsportlet_WAR_smwfactsportlet_parentMessageId").val(e);d.one("#_smwfactsportlet_WAR_smwfactsportlet_body").val(a);if(!themeDisplay.isSignedIn()){window.namespace="_smwfactsportlet_WAR_smwfactsportlet_";window.randomNamespace="raub_";Liferay.Util.openWindow({dialog:{height:460,width:770},id:"_smwfactsportlet_WAR_smwfactsportlet_signInDialog",title:"\u0053\u0069\u0067\u006e\u0020\u0049\u006e",uri:"https://bbp.epfl.ch/nmc-portal/microcircuit?p_p_id=164&p_p_lifecycle=0&p_p_state=pop_up&p_p_mode=view&saveLastPath=false&_164_struts_action=%2Flogin%2Flogin"})}else{_smwfactsportlet_WAR_smwfactsportlet_sendMessage(d)}},["aui-base"]);Liferay.provide(window,"_smwfactsportlet_WAR_smwfactsportlet_sendMessage",function(d,c){var a=AUI();var e=Liferay.Util;d=a.one(d);var b=d.all(".btn-comment");a.io.request(d.attr("action"),{dataType:"json",form:{id:d},on:{complete:function(f,h,g){e.toggleDisabled(b,false)},failure:function(f,h,g){_smwfactsportlet_WAR_smwfactsportlet_showStatusMessage("error","\u0059\u006f\u0075\u0072\u0020\u0072\u0065\u0071\u0075\u0065\u0073\u0074\u0020\u0066\u0061\u0069\u006c\u0065\u0064\u0020\u0074\u006f\u0020\u0063\u006f\u006d\u0070\u006c\u0065\u0074\u0065\u002e")},start:function(){e.toggleDisabled(b,true)},success:function(i,k,j){var g=this.get("responseData");var h=g.exception;if(!h){Liferay.after("smwfactsportlet_WAR_smwfactsportlet:messagePosted",function(l){_smwfactsportlet_WAR_smwfactsportlet_onMessagePosted(g,c)});Liferay.fire("smwfactsportlet_WAR_smwfactsportlet:messagePosted",g)}else{var f="\u0059\u006f\u0075\u0072\u0020\u0072\u0065\u0071\u0075\u0065\u0073\u0074\u0020\u0066\u0061\u0069\u006c\u0065\u0064\u0020\u0074\u006f\u0020\u0063\u006f\u006d\u0070\u006c\u0065\u0074\u0065\u002e";if(h.indexOf("MessageBodyException")>-1){f="\u0050\u006c\u0065\u0061\u0073\u0065\u0020\u0065\u006e\u0074\u0065\u0072\u0020\u0061\u0020\u0076\u0061\u006c\u0069\u0064\u0020\u006d\u0065\u0073\u0073\u0061\u0067\u0065\u002e"}else{if(h.indexOf("NoSuchMessageException")>-1){f="\u0054\u0068\u0065\u0020\u006d\u0065\u0073\u0073\u0061\u0067\u0065\u0020\u0063\u006f\u0075\u006c\u0064\u0020\u006e\u006f\u0074\u0020\u0062\u0065\u0020\u0066\u006f\u0075\u006e\u0064\u002e"}else{if(h.indexOf("PrincipalException")>-1){f="\u0059\u006f\u0075\u0020\u0064\u006f\u0020\u006e\u006f\u0074\u0020\u0068\u0061\u0076\u0065\u0020\u0074\u0068\u0065\u0020\u0072\u0065\u0071\u0075\u0069\u0072\u0065\u0064\u0020\u0070\u0065\u0072\u006d\u0069\u0073\u0073\u0069\u006f\u006e\u0073\u002e"}else{if(h.indexOf("RequiredMessageException")>-1){f="\u0059\u006f\u0075\u0020\u0063\u0061\u006e\u006e\u006f\u0074\u0020\u0064\u0065\u006c\u0065\u0074\u0065\u0020\u0061\u0020\u0072\u006f\u006f\u0074\u0020\u006d\u0065\u0073\u0073\u0061\u0067\u0065\u0020\u0074\u0068\u0061\u0074\u0020\u0068\u0061\u0073\u0020\u006d\u006f\u0072\u0065\u0020\u0074\u0068\u0061\u006e\u0020\u006f\u006e\u0065\u0020\u0069\u006d\u006d\u0065\u0064\u0069\u0061\u0074\u0065\u0020\u0072\u0065\u0070\u006c\u0079\u002e"}}}}_smwfactsportlet_WAR_smwfactsportlet_showStatusMessage("error",f)}}}})},["aui-io"]);Liferay.provide(window,"_smwfactsportlet_WAR_smwfactsportlet_showStatusMessage",function(b,c){var a=AUI();var d=a.one("#_smwfactsportlet_WAR_smwfactsportlet_discussion-status-messages");d.removeClass("alert-error").removeClass("alert-success");d.addClass("alert alert-"+b);d.html(c);d.show()},["aui-base"]);Liferay.provide(window,"raub_subscribeToComments",function(b){var a=AUI();var c=a.one("#_smwfactsportlet_WAR_smwfactsportlet_fm1");var d=c.one("#_smwfactsportlet_WAR_smwfactsportlet_raub_cmd");var e="unsubscribe_from_comments";if(b){e="subscribe_to_comments"}d.val(e);_smwfactsportlet_WAR_smwfactsportlet_sendMessage(c)},["aui-base"]);Liferay.provide(window,"raub_updateMessage",function(d,f){var b=AUI();var e=b.one("#_smwfactsportlet_WAR_smwfactsportlet_fm1");var a=e.one("#_smwfactsportlet_WAR_smwfactsportlet_raub_editReplyBody"+d).val();var c=e.one("#_smwfactsportlet_WAR_smwfactsportlet_messageId"+d).val();if(f){e.one("#_smwfactsportlet_WAR_smwfactsportlet_workflowAction").val("2")}e.one("#_smwfactsportlet_WAR_smwfactsportlet_raub_cmd").val("update");e.one("#_smwfactsportlet_WAR_smwfactsportlet_messageId").val(c);e.one("#_smwfactsportlet_WAR_smwfactsportlet_body").val(a);_smwfactsportlet_WAR_smwfactsportlet_sendMessage(e)},["aui-base"]);</script> </div> </div> </div> <script type="text/javascript">AUI().use("aui-toggler","liferay-store",function(a){(function(){var b;Liferay.component("_smwfactsportlet_WAR_smwfactsportlet_comments-panel",function(){if(!b){b=new a.TogglerDelegate({animated:true,closeAllOnExpand:false,container:"#comments-panel",content:".toggler-content",header:".toggler-header",on:{"toggler:expandedChange":function(d){var f=d.target.get("header");var c=f.getData("persist-id");if(c&&!d.silent){var e={};e[c]=d.newVal?"open":"closed";Liferay.Store(e)}}},transition:{duration:0.3}})}return b});Liferay.component("_smwfactsportlet_WAR_smwfactsportlet_comments-panel")})()});</script> <script type="text/javascript">AUI().use(" aui-toggler","aui-base",function(a){(function(){$(function(){if(location.hash.indexOf("_message_")!=-1){$("#comments-panel .accordion-toggle").click()}})})()});</script> <script>$(smwEquationTooltip.init);$(function(){function a(b){$(".smw-active-tab video").each(function(){if(b){if(this.played.length==0){this.load()}this.play()}else{this.pause()}})}a(true);$(".smw-tab-selector").click(function(){var b=$(this);a(false);$(".smw-tab-selector.active").removeClass("active");b.addClass("active");$(".smw-active-tab").removeClass("smw-active-tab");$("."+b.data("tab")).addClass("smw-active-tab");a(true);return false})});</script> <script src="./assets/morphview.js"></script> </div> <!-- content --> 

## Remove comment section

In [9]:
def replace_using_regexp(old_re, new_re):
    base_folder = '/Users/antonel/Desktop/nmc-portal'

    folders_to_skip = [
        ".git",
        ".DS_Store",
        ".ipynb_checkpoints",
        "widget",
    ]
    for root, dirs, files in os.walk(base_folder):

        if any(x in root for x in folders_to_skip):
#             print(f'skipping folder: {root}')
            continue

        for file in files:
            if not file.endswith('.html'):
                continue
 
            path = f'{root}/{file}'
            print(path)

            with open(path, 'r+') as f:
                html = f.read()
                new_html = change_paths(old_re, new_re, html)
                f.seek(0)
                f.write(new_html)
                f.truncate()

In [ ]:
# old_re = r'<div class="accordion-group lfr-panel lfr-panel-extended" id="comments-panel">.+.click.+?<\/script>'
old_re = r'<div class="accordion-group lfr-panel lfr-panel-extended" id="comments-panel">.+.click\(\).+?<\/script>'
new_re = ''
replace_using_regexp(old_re, new_re)

## Remove errors for literature consistency
- barplot
- Liferay is not defined

In [ ]:
old_re = r'\$\(document\)\.ready\(function \(\) { barplot\.init\(values\) }\);'
new_re = ''
replace_using_regexp(old_re, new_re)

old_re = r'<script type="text\/javascript">Liferay\.Portlet\.list = \[.+\];<\/script>'
new_re = ''
replace_using_regexp(old_re, new_re)

## Add Privacy Policy 

In [10]:
# Add in the footer
old_re = '>Terms of use</a>'
new_re = '>Terms of use</a> <a href="https://go.epfl.ch/privacy-policy">Privacy Policy</a>'
replace_using_regexp(old_re, new_re)

/Users/antonel/Desktop/nmc-portal/howtocite.html
/Users/antonel/Desktop/nmc-portal/videos.html
/Users/antonel/Desktop/nmc-portal/copyright.html
/Users/antonel/Desktop/nmc-portal/morphological-properties.html
/Users/antonel/Desktop/nmc-portal/contact.html
/Users/antonel/Desktop/nmc-portal/microcircuit.html
/Users/antonel/Desktop/nmc-portal/experimental-data.html
/Users/antonel/Desktop/nmc-portal/molecular-properties.html
/Users/antonel/Desktop/nmc-portal/downloads.html
/Users/antonel/Desktop/nmc-portal/gallery.html
/Users/antonel/Desktop/nmc-portal/glossary.html
/Users/antonel/Desktop/nmc-portal/images.html
/Users/antonel/Desktop/nmc-portal/anatomical-circuit-properties.html
/Users/antonel/Desktop/nmc-portal/contributors.html
/Users/antonel/Desktop/nmc-portal/welcome.html
/Users/antonel/Desktop/nmc-portal/tools.html
/Users/antonel/Desktop/nmc-portal/physiological-circuit-properties.html
/Users/antonel/Desktop/nmc-portal/literature-consistency.html
/Users/antonel/Desktop/nmc-portal/ackno

In [ ]:
# Add new page in About
old_re = '<li><a href="copyright.html">Copyright</a></li>'
new_re = '<li><a href="copyright.html">Copyright</a></li> <li><a href="https://go.epfl.ch/privacy-policy">Privacy Policy</a></li>'
replace_using_regexp(old_re, new_re)